In [ ]:
### SETTINGS ####
DATA_FILE_PATH = 'Historical Data for Forecasting.csv'
LAST_DAYS = 100
SIMULATION_DAYS = 14

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
import datetime
register_matplotlib_converters()
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['lines.linewidth'] = 1.5
darkgrey = '#3A3A3A'
lightgrey = '#414141'
barblue = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]
plt.rcParams['text.color'] = darkgrey
plt.rcParams['axes.labelcolor'] = darkgrey
plt.rcParams['xtick.color'] = lightgrey
plt.rcParams['ytick.color'] = lightgrey

# Forecasting Feasible Scope with a Monte Carlo Simulation 'How Many'

## Context

At our planning meeting, the team wants to know how many items can be completed in a given time frame (14 day sprint). Before we make a commitment for a scope and delivery date, we have to forecast the probability to complete the items in time.

## Idea

To understand the current delivery capability, we tracked our throughput and cycle times of our items. We can use this data to forecast future throughput. 

The data points span without date boundaries; i.e., all team data (only) is captured. We will then summarize over the past `P` days.

In [ ]:
def datesWithoutTime(item):
    item['Closed Date'] = datetime.datetime.strptime(item['Closed Date'].strftime('%Y-%m-%d'), '%Y-%m-%d').date()
    return item

kanban_data = pd.read_csv(
    DATA_FILE_PATH, usecols=['Closed Date', 'Work Item Type'], parse_dates=['Closed Date']
).dropna().transform(datesWithoutTime, 'columns')
kanban_data.head(1)

## Analysis

Based on the past throughput per day a forecast can be created with a Monte Carlo simulation. Throughput is the number of total items completed per day.

### Calculate Throughput
Therefore, we sum up the completed items per day and add the missing dates with zero throughput. We plot the data of the throughput per day to get a brief overview of the result.

In [ ]:
throughput = pd.crosstab(
    kanban_data['Closed Date'], kanban_data['Work Item Type'], colnames=[None]).reset_index()
throughput['Throughput'] = (throughput.Bug or 0) + (throughput['User Story'] or 0)
date_range = pd.date_range(
    start=throughput['Closed Date'].min(), end=throughput['Closed Date'].max())
throughput = throughput.set_index('Closed Date').reindex(
    date_range).fillna(0).astype(int).rename_axis('Date')
throughput_per_week = pd.DataFrame(
    throughput['Throughput'].resample('W-Mon').sum()).reset_index()
ax = throughput_per_week.plot(
    x='Date', y='Throughput', linewidth=2.5, figsize=(14, 3), legend=None)
ax.set_title("Throughput per Week", loc='left', fontdict={
             'fontsize': 18, 'fontweight': 'semibold'})
ax.set_xlabel('')
ax.set_ylabel('Items Completed')
ax.axhline(y=0, color=lightgrey, alpha=.5);

### Run Monte Carlo Simulation 'How Many'
Based on the throughput data we simulate multiple times how many items can be completed in the given time span. Before we run the simulation we set the configuration values:
* Date range of data basis (last `P` days)
* Number of items to simulate.
* Number of simulations to run (Recommendation: >= 10000).

We plot the simulation results to get a brief overview of distribution of total items completed in the given timespan.

In [ ]:
SIMULATIONS = 10000
dataset = throughput[['Throughput']].tail(LAST_DAYS).reset_index(drop=True)
samples = [dataset.sample(n=SIMULATION_DAYS, replace=True).sum(
).Throughput for i in range(SIMULATIONS)]
samples = pd.DataFrame(samples, columns=['Items'])
distribution = samples.groupby(['Items']).size().reset_index(name='Frequency')
plt.figure(figsize=(28, 7))
ax = sns.barplot(x='Items', y='Frequency', data=distribution, color=barblue)
ax.set_title(f"Distribution of Monte Carlo Simulation 'How Many' ({SIMULATIONS} Runs)", loc='left',
             fontdict={'size': 18, 'weight': 'semibold'})
ax.set_xlabel(f"Total Items Completed in {SIMULATION_DAYS} Days")
ax.set_ylabel('Frequency')
ax.axhline(y=SIMULATIONS*0.001, color=darkgrey, alpha=.5);

### Analysis of the Probabilities of Occurrence
We determine the probability for each number of completed items by cumulating the frequency in the simulations. We plot the probability for each number of completed items and indicate the percentiles 70%, 85%, and 95%.

In [ ]:
distribution = distribution.sort_index(ascending=False)
distribution['Probability'] = 100 * \
    distribution.Frequency.cumsum()/distribution.Frequency.sum()
plt.figure(figsize=(28, 10))
ax = sns.barplot(x='Items', y='Probability', data=distribution, color=barblue)
ax.text(x=-1.4, y=118,
        s=f"Probabilities of Completing a Scope in {SIMULATION_DAYS} Days", fontsize=18, fontweight='semibold')
ax.text(x=-1.4, y=110,
        s=f"Based on a Monte Carlo Simulations ({SIMULATIONS} Runs) with data of last {LAST_DAYS} days", fontsize=16)
ax.set_ylabel('Confidence')
ax.set_xlabel('Total Items Completed')
ax.axhline(y=0.5, color=darkgrey, alpha=.5)
ax.axhline(y=70, color=darkgrey, linestyle='--')
ax.axhline(y=85, color=darkgrey, linestyle='--')
ax.axhline(y=95, color=darkgrey, linestyle='--')
label_xpos = distribution['Items'].max()-2
ax.text(y=70, x=label_xpos, s=f'70%% (%d+ Items)' % samples.Items.quantile(0.3),
        va='center', ha='center', backgroundcolor='#F0F0F0')
ax.text(y=85, x=label_xpos, s=f'85%% (%d+ Items)' % samples.Items.quantile(0.15),
        va='center', ha='center', backgroundcolor='#F0F0F0')
ax.text(y=95, x=label_xpos, s=f'95%% (%d+ Items)' % samples.Items.quantile(0.05),
        va='center', ha='center', backgroundcolor='#F0F0F0')

## Conclusion

For `P` number of days, we see the number of completed items along with their respective confidence level; with 70% being risky and 95% being confident.